<a href="https://colab.research.google.com/github/creativesar/Rock-paper-scissors/blob/main/Rock_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import os

# ASCII art for game elements
ROCK_ART = """
    _______
---'   ____)
      (_____)
      (_____)
      (____)
---.__(___)
"""

PAPER_ART = """
    _______
---'   ____)____
          ______)
          _______)
         _______)
---.__________)
"""

SCISSORS_ART = """
    _______
---'   ____)____
          ______)
       __________)
      (____)
---.__(___)
"""

# Colors for terminal text
class Colors:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

def clear_screen():
    """Clear the terminal screen."""
    os.system('cls' if os.name == 'nt' else 'clear')

def get_user_choice():
    """Get the user's choice of rock, paper, or scissors."""
    print(f"{Colors.YELLOW}Choose your weapon!{Colors.RESET}")
    print(f"{Colors.BLUE}1. Rock{Colors.RESET} {ROCK_ART}")
    print(f"{Colors.GREEN}2. Paper{Colors.RESET} {PAPER_ART}")
    print(f"{Colors.RED}3. Scissors{Colors.RESET} {SCISSORS_ART}")

    while True:
        user_input = input(f"{Colors.BOLD}Enter your choice (1, 2, 3 or rock, paper, scissors): {Colors.RESET}").lower()

        if user_input in ["1", "rock"]:
            return "rock"
        elif user_input in ["2", "paper"]:
            return "paper"
        elif user_input in ["3", "scissors"]:
            return "scissors"
        else:
            print(f"{Colors.RED}Invalid choice. Try again!{Colors.RESET}")

def get_computer_choice():
    """Generate a random choice for the computer with dramatic effect."""
    print(f"{Colors.PURPLE}Computer is thinking...{Colors.RESET}")
    for _ in range(3):
        print(".", end="", flush=True)
        time.sleep(0.3)
    print()
    return random.choice(["rock", "paper", "scissors"])

def get_art_for_choice(choice):
    """Return ASCII art for a given choice."""
    if choice == "rock":
        return ROCK_ART
    elif choice == "paper":
        return PAPER_ART
    else:
        return SCISSORS_ART

def determine_winner(user_choice, computer_choice):
    """Determine the winner based on the choices."""
    if user_choice == computer_choice:
        return "tie", f"{Colors.YELLOW}It's a tie! Great minds think alike!{Colors.RESET}"

    if (user_choice == "rock" and computer_choice == "scissors") or \
       (user_choice == "paper" and computer_choice == "rock") or \
       (user_choice == "scissors" and computer_choice == "paper"):
        return "user", f"{Colors.GREEN}You win! {user_choice.capitalize()} beats {computer_choice}!{Colors.RESET}"
    else:
        return "computer", f"{Colors.RED}Computer wins! {computer_choice.capitalize()} beats {user_choice}!{Colors.RESET}"

def play_game(scores):
    """Play a single round of rock, paper, scissors."""
    clear_screen()
    print(f"{Colors.BOLD}{Colors.PURPLE}===== ROCK, PAPER, SCISSORS SHOWDOWN ====={Colors.RESET}")
    print(f"Current Score - You: {scores['user']} | Computer: {scores['computer']} | Ties: {scores['tie']}\n")

    user_choice = get_user_choice()
    computer_choice = get_computer_choice()

    print(f"\n{Colors.BLUE}You chose: {user_choice.upper()}{Colors.RESET}")
    print(get_art_for_choice(user_choice))

    print(f"{Colors.RED}Computer chose: {computer_choice.upper()}{Colors.RESET}")
    print(get_art_for_choice(computer_choice))

    winner, result_message = determine_winner(user_choice, computer_choice)
    scores[winner] += 1

    print("\n" + result_message)
    print(f"\nUpdated Score - You: {scores['user']} | Computer: {scores['computer']} | Ties: {scores['tie']}")

def main():
    """Main function to run the game with replay option."""
    scores = {"user": 0, "computer": 0, "tie": 0}

    while True:
        play_game(scores)

        print(f"\n{Colors.YELLOW}Do you want to play again?{Colors.RESET}")
        play_again = input(f"{Colors.BOLD}(yes/no): {Colors.RESET}").lower()

        if play_again != "yes":
            clear_screen()
            print(f"{Colors.PURPLE}===== FINAL SCORE ====={Colors.RESET}")
            print(f"You: {scores['user']} | Computer: {scores['computer']} | Ties: {scores['tie']}")

            if scores['user'] > scores['computer']:
                print(f"{Colors.GREEN}Congratulations! You are the ultimate Rock, Paper, Scissors champion!{Colors.RESET}")
            elif scores['computer'] > scores['user']:
                print(f"{Colors.RED}The computer has defeated you. Better luck next time!{Colors.RESET}")
            else:
                print(f"{Colors.YELLOW}It's a tie overall! You and the computer are evenly matched!{Colors.RESET}")

            print(f"{Colors.BLUE}Thanks for playing!{Colors.RESET}")
            break

if __name__ == "__main__":
    main()

===== ROCK, PAPER, SCISSORS SHOWDOWN =====
Current Score - You: 0 | Computer: 0 | Ties: 0

Choose your weapon!
1. Rock 
    _______
---'   ____)
      (_____)
      (_____)
      (____)
---.__(___)

2. Paper 
    _______
---'   ____)____
          ______)
          _______)
         _______)
---.__________)

3. Scissors 
    _______
---'   ____)____
          ______)
       __________)
      (____)
---.__(___)

Enter your choice (1, 2, 3 or rock, paper, scissors): 1
Computer is thinking...
...

You chose: ROCK

    _______
---'   ____)
      (_____)
      (_____)
      (____)
---.__(___)

Computer chose: SCISSORS

    _______
---'   ____)____
          ______)
       __________)
      (____)
---.__(___)


You win! Rock beats scissors!

Updated Score - You: 1 | Computer: 0 | Ties: 0

Do you want to play again?
